# Limiting Agent Recursion Depth

There are several practical reasons to cap agent iterations:

1. **Preventing infinite loops**: Agents can get stuck in cycles where they repeatedly call the same tool without making progress. This often happens when:
    - The model misinterprets tool outputs and keeps retrying the same approach
    - A tool returns an error the model doesn't know how to handle
    - The prompt or task is ambiguous, causing the agent to loop trying different interpretations

2. **Cost control**: Each iteration involves an LLM call (and potentially API calls to external tools). An agent stuck in a loop can rack up significant token costs quickly. A recursion limit acts as a financial circuit breaker.
3. **Latency bounds**: For user-facing applications, you need predictable response times. An unbounded agent could take minutes or longer to respond. Setting a limit lets you guarantee worst-case latency and provide feedback like "I couldn't complete this task in the allowed steps."
4. **Adversarial robustness**: Malicious prompts can try to trick agents into looping forever. The recursion limit is a safety net against prompt injection attacks that attempt to hijack the agent's execution.
5. **Debugging and observability**: When an agent hits the limit, it's a clear signal something went wrong. You can inspect the intermediate steps to understand why the agent didn't converge on a solution, which is much easier to debug than an agent that runs indefinitely. In practice, most well-designed agent tasks should complete in 3-10 iterations. If you're consistently hitting limits, it usually indicates a problem with your tool definitions, prompts, or task decomposition rather than a need for more iterations.

## Defining recursion limit
- In LangChain >= 1.0, `create_agent` builds a LangGraph-based agent, so iteration limits are controlled via `recursion_limit`
- The default `recursion_limit` is 25. 
- Each agent "iteration" (model call → tool execution) typically consumes 2 graph steps
- If you want N tool-calling iterations, set `recursion_limit` to roughly `1 + 2 * num_tool_calls + 1`

There are two ways to set this:
1. Set as a default on the agent using .with_config():

    ```python
    agent = create_agent(
        model="your-model",
        tools=tools,
        system_prompt="You are a helpful assistant."
    ).with_config({"recursion_limit": 50})

    result = agent.invoke({"messages": [{"role": "user", "content": "..."}]})
    ```

2. Pass at runtime when invoking:

    ```python
    agent = create_agent(model="your-model", tools=tools)

    result = agent.invoke(
        {"messages": [{"role": "user", "content": "..."}]},
        config={"recursion_limit": 50}
    )
    ```

In [ ]:
from hashlib import sha256

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain.tools import tool
from loguru import logger

from chain_reaction.config import APIKeys, ModelBehavior, ModelName

In [ ]:
# Define a custom tool to hash a string (the agent cant already know the answer to a hash)
@tool
def hash_string(s: str) -> str:
    """Generate a SHA-256 hash of the input string."""
    logger.debug(f"Hashing string: {s}")
    return sha256(s.encode("utf-8")).hexdigest()


hash_string.invoke("ABCdef")

In [ ]:
# Initialize the agent with the hashing tool
agent = create_agent(
    model=init_chat_model(
        model=ModelName.CLAUDE_HAIKU,
        timeout=None,
        max_retries=2,
        api_key=APIKeys().anthropic,
        **ModelBehavior.deterministic().model_dump(),
    ),
    tools=[hash_string],
    system_prompt="""
    You're a helpful assistant that can hash strings using the provided tool.
    Use the provided hashing tool to answer user questions accurately.
    """,
)

In [ ]:
# Invoke the agent with a prompt that requires 2 steps of hashing, using default recursion limits
agent.invoke({
    "messages": [
        HumanMessage(content="Hash the string 'Hello, World!'. Then add that hash to '12345' and hash the result.")
    ]
})

In [ ]:
# Now invoke the agent with a prompt that requires 3 steps of hashing, but set a custom recursion limit to ~2 tool calls
max_tool_calls = 2

agent.invoke(
    input={
        "messages": [
            HumanMessage(
                content=(
                    "Hash the string 'Hello, World!'."
                    " Then add that hash to '12345' and hash the result."
                    " Then add that hash to 'ABC' and hash the result."
                )
            )
        ]
    },
    config={"recursion_limit": 1 + 2 * max_tool_calls + 1},
)